# Calling Necessary Libraries AND Getting Today's Date

In [18]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser
from scipy.stats import poisson
import warnings

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Get today's date
given_date = "2025-02-21" #year-month-day
threshold = 70 #threshold for percentages to highlight in final dataframe (between 1 and 100)

wanted_leagues = ['austria', 'australia', 'belgium', 'brazil', 'colombia','denmark', 
                  'england', 'england2', 'england3', 'england4', 'england5', 'france', 'france2',
                  'germany', 'germany2', 'greece', 'italy', 'italy2',
                   'mexico2', 'netherlands', 'norway','poland', 'portugal', 'portugal2',
                   'saudiarabia','scotland', 'spain', 'spain2', 'sweden', 'switzerland', 'turkey'] 
#argent, urug, peru, ecuado, usa

In [19]:
#Calculating Days between given date and today
# Today's date
today = datetime.now().date()

# Specific date
specific_date = datetime.strptime(given_date, "%Y-%m-%d").date()

# Calculate the difference in days
difference = specific_date - today

# Add one day to the difference
days_until_specific_date = difference.days + 1

# Scraping Today's Matches and Leagues

In [20]:
URL = "https://www.soccerstats.com/matches.asp?matchday=" + str(days_until_specific_date) + "&listing=2"
page = requests.get(URL)
liqa = []
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="btable")
sth = results.find_all("tr", attrs={'height': '34'})

# Converting Date and Collecting Leagues for Analysis

In [21]:
from dateutil import parser

day_abbreviations = {0: "Mo", 1: "Tu", 2: "We", 3: "Th", 4: "Fr", 5: "Sa", 6: "Su"}
given_date_parsed = parser.parse(given_date)

# Manually format the day to remove leading zeros
day = given_date_parsed.day
month = given_date_parsed.strftime('%b')

# Format the date as "Su 1 Oct" or "Tu 10 Oct" without leading zero for single-digit days
formatted_date = f"{day_abbreviations[given_date_parsed.weekday()]} {day} {month}"
print(formatted_date)

Fr 21 Feb


# Scraping the Web for the League Statistics

In [22]:
final =  pd.DataFrame()
liqa = ''
unique_leagues = wanted_leagues
next_matches = pd.DataFrame()

for i in unique_leagues:
    URL = "https://www.soccerstats.com/results.asp?league=" + i + "&pmtype=bydate"
    page = requests.get(URL)
    liqa = i
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="btable")
    sth = results.find_all("tr", class_="odd")
    sth


    date, league, home, away, ft, ht = [], [], [], [], [],[]
    for i in sth:
        date.append(i.find_all("td", align = 'right')[0].get_text(strip=True))
        league.append(liqa.capitalize())
        home.append(i.find_all("td", align = 'right')[1].get_text(strip=True))
        away.append(i.find("td", align = "left").get_text(strip = True))
        ft.append(i.find_all("td", align = 'center')[0].get_text(strip = True))
        try:
            ht.append(i.find_all("td", align = 'center')[2].get_text(strip = True))
        except IndexError as e:
            ht.append('NA')#print("Last output before error occurred:", i.find_all("td", align = 'center'))

    data = {'Date': date, 'League': league,'Home': home, 'Away': away, 'FT': ft, 'HT': ht}

# Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

# Replace empty strings with NaN
    next_df = df[(df['Date'] == formatted_date) & (df['HT'] == '')]
    next_matches = pd.concat([next_matches, next_df], ignore_index = True)
    df.replace('', pd.NA, inplace=True)

# Drop rows with NaN values
    df_cleaned = df.dropna()

#For Half-Time Results
    hthg, htag = [], []
    for i in df_cleaned['HT']:
        if i == 'NA':
            hthg.append('NA')
            htag.append('NA')
        elif i == '+' or i == '-':
            hthg.append('NA')
            htag.append('NA')
        else:
            try:
                hthg.append(int(i[1]))
                htag.append(int(i[3]))
            except IndexError as e:
                print("Last output before error occurred:", i)



#For Full-Time Results
    hg, ag, tg = [], [], []
    for i in df_cleaned['FT']:
        if len(i) < 5 or ':' in i:
            hg.append('NA')
            ag.append('NA')
            tg.append('NA')
        else:
            try:
                hghg = int(i.split(' - ')[0])
                hg.append(hghg)
                agag = int(i.split(' - ')[1])
                ag.append(agag)
                tg.append(hghg + agag)
            except:
                print(hghg + agag)

    
    df_cleaned['FTHG'], df_cleaned['FTAG'], df_cleaned['FTTG'] = hg, ag, tg
    df_cleaned['HTHG'], df_cleaned['HTAG'] = hthg, htag
    df_cleaned['HTTG'] = df_cleaned['HTHG'] + df_cleaned['HTAG']
    
    final = pd.concat([final, df_cleaned], ignore_index=True)
    
final = final[final['HT'] != 'NA']

# Define current year
current_year = datetime.now().year

# Function to parse and assign the correct year
def parse_date(date_str):
    # Extract day and month
    date_obj = datetime.strptime(date_str[3:], '%d %b')
    # Assign correct year
    full_date = date_obj.replace(year=current_year)
    
    # If the parsed date is already in the future, assign the previous year
    if full_date > datetime.now():
        full_date = full_date.replace(year=current_year - 1)
    
    return full_date

# Apply the function to parse dates
final['Date'] = final['Date'].apply(parse_date)

# Find the latest date for each league
latest_dates = final.groupby('League')['Date'].max().rename('latest_date')

# Merge with the original DataFrame
final = final.merge(latest_dates, on='League')

# Calculate the time difference in days
final['time_diff'] = (final['latest_date'] - final['Date']).dt.days
combined_df = pd.concat([final.head(), final.tail()])
combined_df

,Date,League,Home,Away,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG,latest_date,time_diff
0,2024-08-02,Austria,Grazer AK,Salzburg,2 - 3,(2-3),2.0,3.0,5.0,2.0,3.0,5.0,2025-02-16,198
1,2024-08-03,Austria,Altach,Tirol,1 - 2,(0-1),1.0,2.0,3.0,0.0,1.0,1.0,2025-02-16,197
2,2024-08-03,Austria,Hartberg,LASK Linz,1 - 2,(0-1),1.0,2.0,3.0,0.0,1.0,1.0,2025-02-16,197
3,2024-08-03,Austria,Wolfsberger AC,A. Klagenfurt,4 - 1,(3-0),4.0,1.0,5.0,3.0,0.0,3.0,2025-02-16,197
4,2024-08-04,Austria,BW Linz,Austria Wien,1 - 0,(0-0),1.0,0.0,1.0,0.0,0.0,0.0,2025-02-16,196
6277,2025-02-16,Turkey,Hatayspor,Alanyaspor,1 - 0,(0-0),1.0,0.0,1.0,0.0,0.0,0.0,2025-02-17,1
6278,2025-02-16,Turkey,Eyupspor,Kayserispor,1 - 1,(0-0),1.0,1.0,2.0,0.0,0.0,0.0,2025-02-17,1
6279,2025-02-16,Turkey,Konyaspor,Samsunspor,0 - 1,(0-1),0.0,1.0,1.0,0.0,1.0,1.0,2025-02-17,1
6280,2025-02-16,Turkey,Fenerbahce,Kasimpasa,3 - 1,(1-0),3.0,1.0,4.0,1.0,0.0,1.0,2025-02-17,1
6281,2025-02-17,Turkey,Rizespor,Galatasaray,1 - 2,(0-0),1.0,2.0,3.0,0.0,0.0,0.0,2025-02-17,0


In [23]:
next_leagues = next_matches['League'].unique().tolist()
pd.concat([next_matches.head(), next_matches.tail()])

,Date,League,Home,Away,FT,HT
0,Fr 21 Feb,Australia,Newcastle Jets,Brisbane Roar,08:35,
1,Fr 21 Feb,Belgium,KV Mechelen,Sint-Truiden,19:45,
2,Fr 21 Feb,Denmark,Lyngby,Randers FC,18:00,
3,Fr 21 Feb,England,Leicester City,Brentford,20:00,
4,Fr 21 Feb,England2,Bristol City,Middlesbrough,19:45,
24,Fr 21 Feb,Saudiarabia,Al Nassr,Al Ittifaq,17:00,
25,Fr 21 Feb,Spain,Celta Vigo,Osasuna,20:00,
26,Fr 21 Feb,Spain2,Malaga,Tenerife,19:30,
27,Fr 21 Feb,Turkey,Bodrumspor,Hatayspor,17:00,
28,Fr 21 Feb,Turkey,Eyupspor,Besiktas,17:00,


# Calculating Functions Needed for Dixon-Coles Model

In [24]:
from scipy.optimize import minimize
from scipy.stats import poisson

def rho_correction(x, y, lambda_x, mu_y, rho):
    if x==0 and y==0:
        return 1- (lambda_x * mu_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (mu_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0

def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
    lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
    return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
            np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

def solve_parameters_decay(dataset, half_or_full = 'full', xi=0.001, debug = False, init_vals=None, 
                           options={'disp': True, 'maxiter':100},
                     constraints = [{'type':'eq', 'fun': lambda x: sum(x[:20])-20}] , **kwargs):
    teams = np.sort(dataset['Home'].unique())
    # check for no weirdness in dataset
    away_teams = np.sort(dataset['Away'].unique())
    if not np.array_equal(teams, away_teams):
        raise ValueError("Home Teams Not Equal to Away Teams")
    n_teams = len(teams)
    if init_vals is None:
        # random initialisation of model parameters
        init_vals = np.concatenate((np.random.uniform(0,1,(n_teams)), # attack strength
                                      np.random.uniform(0,-1,(n_teams)), # defence strength
                                      np.array([0,1.0]) # rho (score correction), gamma (home advantage)
                                     ))
        
    def dc_log_like_decay(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma, t, xi=xi):
        lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
        return  np.exp(-xi*t) * (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
                                  np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

    def estimate_paramters(params):
        score_coefs = dict(zip(teams, params[:n_teams]))
        defend_coefs = dict(zip(teams, params[n_teams:(2*n_teams)]))
        rho, gamma = params[-2:]
        if half_or_full == 'full':
            log_like = [dc_log_like_decay(row.FTHG, row.FTAG, score_coefs[row.Home], defend_coefs[row.Home],
                                      score_coefs[row.Away], defend_coefs[row.Away], 
                                      rho, gamma, row.time_diff, xi=xi) for row in dataset.itertuples()]
        elif half_or_full == 'half':
            log_like = [dc_log_like_decay(row.HTHG, row.HTAG, score_coefs[row.Home], defend_coefs[row.Home],
                                      score_coefs[row.Away], defend_coefs[row.Away], 
                                      rho, gamma, row.time_diff, xi=xi) for row in dataset.itertuples()]
        return -sum(log_like)
    opt_output = minimize(estimate_paramters, init_vals, options=options, constraints = constraints)
    if debug:
        # sort of hacky way to investigate the output of the optimisation process
        return opt_output
    else:
        return dict(zip(["attack_"+team for team in teams] + 
                        ["defence_"+team for team in teams] +
                        ['rho', 'home_adv'],
                        opt_output.x))

# Calculating Lambda Values for Dixon-Coles Model

In [25]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
stats_df = pd.DataFrame()
full_time_models = []
half_time_models = []

for league in next_leagues:
    league_df = final[final['League'] == league.capitalize()]
    
    full_time_estimates = solve_parameters_decay(league_df, half_or_full = 'full')
    full_time_models.append(full_time_estimates)

    half_time_estimates = solve_parameters_decay(league_df, half_or_full = 'half')
    half_time_models.append(half_time_estimates)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 294.1694366372032
            Iterations: 34
            Function evaluations: 1027
            Gradient evaluations: 34
Optimization terminated successfully    (Exit mode 0)
            Current function value: 208.70126444982284
            Iterations: 33
            Function evaluations: 991
            Gradient evaluations: 33
Optimization terminated successfully    (Exit mode 0)
            Current function value: 521.9993952265315
            Iterations: 38
            Function evaluations: 1383
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 369.5878103787633
            Iterations: 35
            Function evaluations: 1264
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 260.88683491764147
            Iterations: 34
            Function

In [26]:
full_time_models[-1]

{'attack_Adana Demirspor': 0.6518767777756515,
 'attack_Alanyaspor': 0.8655024301001437,
 'attack_Antalyaspor': 0.9123152312927983,
 'attack_Basaksehir': 1.2858395193218701,
 'attack_Besiktas': 1.1593786053863289,
 'attack_Bodrumspor': 0.38340534089757106,
 'attack_Eyupspor': 1.1631902721434815,
 'attack_Fenerbahce': 1.6731410452597646,
 'attack_Galatasaray': 1.6476046046396255,
 'attack_Gaziantep': 0.9932714628670204,
 'attack_Goztepe': 1.3348907750971184,
 'attack_Hatayspor': 0.7825826826941322,
 'attack_Kasimpasa': 1.3764280864615723,
 'attack_Kayserispor': 0.8730738686201222,
 'attack_Konyaspor': 1.0070658050241696,
 'attack_Rizespor': 0.8873831966780394,
 'attack_Samsunspor': 1.2268603466334138,
 'attack_Sivasspor': 0.9812615978872257,
 'attack_Trabzonspor': 1.2795990089074165,
 'defence_Adana Demirspor': -0.48467065768746714,
 'defence_Alanyaspor': -1.0501370092672861,
 'defence_Antalyaspor': -0.6616911893220829,
 'defence_Basaksehir': -0.9173076687678586,
 'defence_Besiktas': -1

# Calculating Probability Matrices for Half/Full Time

In [27]:
#First Function needs work to make it more understandable and a df rather than matrix!
def dixon_coles_simulate_match(params_dict, homeTeam, awayTeam, max_goals=10):
    team_avgs = [np.exp(params_dict['attack_'+homeTeam] + params_dict['defence_'+awayTeam] + params_dict['home_adv']),
                 np.exp(params_dict['defence_'+homeTeam] + params_dict['attack_'+awayTeam])]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in team_avgs]
    output_matrix = np.outer(np.array(team_pred[0]), np.array(team_pred[1]))
    correction_matrix = np.array([[rho_correction(home_goals, away_goals, team_avgs[0],
                                                   team_avgs[1], params_dict['rho']) for away_goals in range(2)]
                                   for home_goals in range(2)])
    output_matrix[:2,:2] = output_matrix[:2,:2] * correction_matrix
    return output_matrix

full_time_matrices = []
half_time_matrices = []

for i in range(len(next_matches)):
    my_league = next_matches['League'].iloc[i]
    league_index = next_leagues.index(my_league)
    ft_match_score_matrix = dixon_coles_simulate_match(full_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 8)
    ht_match_score_matrix = dixon_coles_simulate_match(half_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 4)
    full_time_matrices.append(ft_match_score_matrix)
    half_time_matrices.append(ht_match_score_matrix)

full_time_matrices[0]

array([[3.83861098e-02, 6.45976852e-02, 3.90736236e-02, 1.73169874e-02,
        5.75601953e-03, 1.53060160e-03, 3.39172646e-04, 6.44217752e-05,
        1.07066254e-05],
       [8.49212598e-02, 9.93476038e-02, 6.99144193e-02, 3.09852788e-02,
        1.02992435e-02, 2.73870483e-03, 6.06881482e-04, 1.15269857e-04,
        1.91573606e-05],
       [7.07668728e-02, 9.40892296e-02, 6.25489214e-02, 2.77209735e-02,
        9.21421613e-03, 2.45018174e-03, 5.42946398e-04, 1.03126155e-04,
        1.71391289e-05],
       [4.22077125e-02, 5.61179405e-02, 3.73062534e-02, 1.65337090e-02,
        5.49566443e-03, 1.46136974e-03, 3.23831259e-04, 6.15078626e-05,
        1.02223455e-05],
       [1.88805608e-02, 2.51029522e-02, 1.66880161e-02, 7.39593975e-03,
        2.45834755e-03, 6.53707074e-04, 1.44857786e-04, 2.75139986e-05,
        4.57270971e-06],
       [6.75659601e-03, 8.98334048e-03, 5.97197214e-03, 2.64671041e-03,
        8.79744059e-04, 2.33935562e-04, 5.18387958e-05, 9.84615736e-06,
        1.6

# Calculating Probabilities of Dixon-Coles Model

In [28]:
ft1, ftx, ft2, ft_score = [], [], [], []
over_15, over_25, under_35, under_45, btts = [], [], [], [], []
ht1, htx, ht2, ht_score, ht_over05, ht_under15 = [], [], [], [], [], []
ho05, ao05, ho15, ao15, hu25, au25 = [], [], [], [], [], []

# Helper function to calculate total goals for each score
def total_goals(i, j):
    return i + j

for i in range(len(next_matches)):
    my_matrix = full_time_matrices[i]
    ht_matrix = half_time_matrices[i]

    ft1.append(round(np.sum(np.tril(my_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    ftx.append(round(np.sum(np.diag(my_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ft2.append(round(np.sum(np.triu(my_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)
    
    max_score = np.unravel_index(np.argmax(my_matrix), my_matrix.shape) # Find the index of the maximum score
    home_goals, away_goals = max_score
    ft_score.append(f"{home_goals}-{away_goals}") # Format the score as 'home-away'

    # Calculate the probabilities
    over_15.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 1.5]) * 100, 2))
    over_25.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 2.5]) * 100, 2))
    under_35.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 3.5]) * 100, 2))
    under_45.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 4.5]) * 100, 2))

    # Calculate BTTS (both teams to score and goals != 0)
    btts.append(round(np.sum([my_matrix[i, j] for i in range(1, my_matrix.shape[0]) for j in range(1, my_matrix.shape[1])]) * 100, 2)) 

    # Calculate statistics for Half Time
    ht1.append(round(np.sum(np.tril(ht_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    htx.append(round(np.sum(np.diag(ht_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ht2.append(round(np.sum(np.triu(ht_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)

    ht_max_score = np.unravel_index(np.argmax(ht_matrix), ht_matrix.shape) # Find the index of the maximum score
    ht_hogs, ht_awgs = ht_max_score
    ht_score.append(f"{ht_hogs}-{ht_awgs}") # Format the score as 'home-away'

    ht_over05.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) > 0.5]) * 100, 2))   
    ht_under15.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) < 1.5]) * 100, 2)) 

    ho05.append(round(np.sum(my_matrix[1:,:]) * 100, 2))
    ao05.append(round(np.sum(my_matrix[:,1:]) * 100, 2))
    ho15.append(round(np.sum(my_matrix[2:,:]) * 100, 2))
    ao15.append(round(np.sum(my_matrix[:,2:]) * 100, 2))
    hu25.append(round(np.sum(my_matrix[:3,:]) * 100, 2))
    au25.append(round(np.sum(my_matrix[:,:3]) * 100, 2))
    

# Combine lists into a DataFrame
final_results = pd.DataFrame({
    'League': next_matches['League'], 'Home': next_matches['Home'], 'Away': next_matches['Away'],
    'FT1': ft1, 'FTX': ftx, 'FT2': ft2, 'FTR': ft_score,
    'DC1X': [x + y for x, y in zip(ft1, ftx)], 'DC12': [x + y for x, y in zip(ft1, ft2)], 'DCX2': [x + y for x, y in zip(ftx, ft2)],
    '1.5O': over_15, '2.5O': over_25, '3.5U': under_35, '4.5U': under_45, 'BTTS': btts,
    'HT1': ht1, 'HTX': htx, 'HT2': ht2, 'HTR': ht_score,
    'HTDC1X': [x + y for x, y in zip(ht1, htx)], 'HTDC12': [x + y for x, y in zip(ht1, ht2)], 'HTDCX2': [x + y for x, y in zip(htx, ht2)],
    'HT0.5O': ht_over05, 'HT1.5U': ht_under15, 'H0.5O':ho05, 'A0.5O':ao05, 'H1.5O':ho15, 'A1.5O':ao15, 'H2.5U':hu25, 'A2.5U':au25
})

# Function to highlight values higher than threshold
def highlight_values(value):
    if isinstance(value, str):
        return ''  # Return empty string for NaN values
    elif value > threshold:
    #color = 'red'
        return 'background-color: red'
    else:
        return ''

# Apply the style
with pd.option_context('display.precision', 2):
    styled_df = final_results.style.applymap(highlight_values)
styled_df.to_excel(given_date + ".xlsx", index = False)
# Display the styled DataFrame
from IPython.display import display, HTML
display(styled_df)

,League,Home,Away,FT1,FTX,FT2,FTR,DC1X,DC12,DCX2,1.5O,2.5O,3.5U,4.5U,BTTS,HT1,HTX,HT2,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O,HT1.5U,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U
0,Australia,Newcastle Jets,Brisbane Roar,48.870000,21.950000,29.170000,1-1,70.820000,78.040000,51.120000,81.200000,60.280000,62.060000,79.490000,60.660000,22.310000,43.190000,34.410000,0-0,65.500000,56.720000,77.600000,68.640000,66.360000,83.280000,73.530000,53.390000,38.360000,73.350000,85.020000
1,Belgium,KV Mechelen,Sint-Truiden,69.790000,18.480000,11.550000,2-1,88.270000,81.340000,30.030000,89.910000,72.020000,48.500000,67.870000,62.450000,44.060000,31.500000,24.070000,1-0,75.560000,68.130000,55.570000,80.620000,54.860000,92.900000,65.690000,74.430000,29.060000,50.070000,90.420000
2,Denmark,Lyngby,Randers FC,13.700000,29.880000,56.420000,0-1,43.580000,70.120000,86.300000,57.450000,29.560000,87.510000,95.610000,31.750000,7.170000,48.340000,44.410000,0-0,55.510000,51.580000,92.750000,56.410000,79.300000,41.480000,74.380000,10.120000,39.490000,98.270000,84.260000
3,England,Leicester City,Brentford,16.220000,20.180000,63.510000,1-2,36.400000,79.730000,83.690000,88.120000,69.350000,51.860000,70.950000,63.820000,13.540000,30.110000,55.760000,0-1,43.650000,69.300000,85.870000,78.210000,57.630000,69.120000,90.900000,32.880000,69.220000,88.360000,56.790000
4,England2,Bristol City,Middlesbrough,44.260000,26.920000,28.820000,1-1,71.180000,73.080000,55.740000,76.420000,51.250000,70.850000,85.910000,55.450000,28.590000,46.400000,24.980000,0-0,74.990000,53.570000,71.380000,62.350000,75.520000,78.380000,69.690000,45.270000,33.510000,80.080000,88.080000
5,England2,Burnley,Sheffield Wed,72.980000,22.390000,4.630000,1-0,95.370000,77.610000,27.020000,57.420000,29.980000,87.250000,95.480000,18.280000,48.640000,46.300000,4.960000,0-0,94.940000,53.600000,51.260000,56.650000,79.880000,81.060000,22.020000,49.550000,2.630000,76.660000,99.780000
6,France,Rennes,Reims,46.320000,30.570000,23.110000,1-1,76.890000,69.430000,53.680000,73.290000,45.340000,75.990000,89.260000,51.180000,30.660000,37.140000,32.070000,0-0,67.800000,62.730000,69.210000,74.900000,62.040000,77.460000,63.170000,43.880000,26.390000,81.130000,91.990000
7,France2,Grenoble,Guingamp,38.310000,21.470000,40.220000,0-1,59.780000,78.530000,61.690000,72.930000,50.600000,71.440000,86.310000,52.650000,20.800000,44.340000,34.810000,0-0,65.140000,55.610000,79.150000,63.230000,76.170000,73.510000,74.590000,38.320000,39.780000,85.050000,84.070000
8,France2,Martigues,Annecy,19.530000,22.020000,58.440000,0-1,41.550000,77.970000,80.460000,62.520000,37.780000,81.890000,92.730000,37.140000,18.570000,47.460000,33.950000,0-0,66.030000,52.520000,81.410000,59.050000,79.670000,50.480000,77.670000,15.680000,44.190000,96.550000,80.900000
9,France2,Paris FC,Troyes,56.800000,23.500000,19.700000,1-0,80.300000,76.500000,43.200000,58.520000,33.560000,84.880000,94.320000,34.020000,34.140000,46.140000,19.680000,0-0,80.280000,53.820000,65.820000,60.840000,78.230000,75.080000,48.100000,40.460000,14.060000,83.600000,97.100000
